In [1]:
from tinyagents import chainable, respond

@chainable(
    node_name="main_agent",
    ray_options={
        "num_replicas": 2
    }
)
class Agent:
    name: str = "main_agent"

    def __init__(self):
        pass

    def run(self, inputs: str):
        return "Hello! I was provided with some tool outputs: " + str(inputs)
    
    def output_handler(self, output):
        return respond(output)

@chainable(
    node_name="tool1",
    ray_options={
        "num_replicas": 1
    }
)
class Tool1:
    def run(self, inputs):
        return "Tool 1 has been triggered."

@chainable(
    node_name="tool2",
    ray_options={
        "num_replicas": 1
    }
)
class Tool2:
    def run(self, inputs):
        return f"Tool 2 has been triggered."

/Users/adamhedib/Library/Caches/pypoetry/virtualenvs/tinyagents-JQO3J91T-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-17 09:19:37,128	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-17 09:19:37,786	WARNING api.py:346 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:37,787	WARNING api.py:397 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.


In [2]:
branch1 = Tool1() / Tool2()

def branch_router(inputs) -> str:
    if inputs.startswith("blue"):
        return Tool1.name
    
    return Tool2.name
    
branch1.bind_router(branch_router)

ConditionalBranch(tool1, tool2)

In [3]:
graph = branch1 | Agent()
str(graph)

'ConditionalBranch(tool1, tool2) -> main_agent'

In [4]:
# Run without using Ray
runner = graph.compile()
inputs = "blue!"
y = await runner.ainvoke(inputs)
y

blue!

 > Running node: conditional_branch_tool1-tool2

	Input: blue!


 > Running node: tool1

	Input: blue!

	Output (tool1): {
  "content": "Tool 1 has been triggered.",
  "action": null,
  "ref": null
}
	Output (conditional_branch_tool1-tool2): {
  "content": "Tool 1 has been triggered.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 1 has been triggered.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 1 has been triggered.'

In [5]:
runner.invoke("yellow!")


 > Running node: conditional_branch_tool1-tool2

	Input: yellow!


 > Running node: tool2

	Input: yellow!

	Output (tool2): {
  "content": "Tool 2 has been triggered.",
  "action": null,
  "ref": null
}
	Output (conditional_branch_tool1-tool2): {
  "content": "Tool 2 has been triggered.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 2 has been triggered.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 2 has been triggered.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 2 has been triggered.'

In [6]:
# Run using Ray
runner = graph.compile(use_ray=True, verbose=False)
runner

2024-10-17 09:19:46,584	WARNING api.py:346 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:46,585	WARNING api.py:397 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:46,587	WARNING api.py:346 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:46,589	WARNING api.py:397 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:46,591	WARNING api.py:346 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-10-17 09:19:46,592	WARNING api.py:397 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.


In [7]:
from ray import serve

# we can now serve it using Ray Serve

app = serve.run(runner, name="my_application")

2024-10-17 09:19:48,029	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-10-17 09:19:49,948	INFO handle.py:126 -- Created DeploymentHandle '4e3fci5j' for Deployment(name='tool1', app='my_application').
2024-10-17 09:19:49,948	INFO handle.py:126 -- Created DeploymentHandle 'zxgr5fl1' for Deployment(name='tool2', app='my_application').
2024-10-17 09:19:49,949	INFO handle.py:126 -- Created DeploymentHandle 't4qrxibn' for Deployment(name='main_agent', app='my_application').
2024-10-17 09:19:49,949	INFO handle.py:126 -- Created DeploymentHandle 'gwoswrgg' for Deployment(name='tool1', app='my_application').
2024-10-17 09:19:49,950	INFO handle.py:126 -- Created DeploymentHandle 'f1cf4at1' for Deployment(name='tool2', app='my_application').
2024-10-17 09:19:49,950	INFO handle.py:126 -- Created DeploymentHandle '71amo9vj' for Deployment(name='main_agent', app='my_application').
2024-10-17 09:19:49,951	INFO handle.py:126 -- Created Deployment

(ServeReplica:my_application:tool1 pid=32675) INFO 2024-10-17 09:19:53,962 my_application_tool1 mp0j7vsa 5155d006-c883-4ca4-aa20-ad8eafd89ed7 replica.py:373 - AINVOKE OK 1.6ms
(ServeReplica:my_application:main_agent pid=32677) INFO 2024-10-17 09:19:53,974 my_application_main_agent e9tf3qs6 5155d006-c883-4ca4-aa20-ad8eafd89ed7 replica.py:373 - AINVOKE OK 1.8ms
(ServeReplica:my_application:runner pid=32679) INFO 2024-10-17 09:19:53,974 my_application_runner 04ga38ej 5155d006-c883-4ca4-aa20-ad8eafd89ed7 replica.py:373 - AINVOKE OK 57.3ms
(ServeReplica:my_application:tool2 pid=32676) INFO 2024-10-17 09:20:04,523 my_application_tool2 pco21ch0 13a82d12-a956-4d6b-8ea2-429a7a945312 replica.py:373 - AINVOKE OK 2.3ms
(ServeReplica:my_application:runner pid=32679) INFO 2024-10-17 09:20:04,535 my_application_runner 04ga38ej 13a82d12-a956-4d6b-8ea2-429a7a945312 replica.py:373 - AINVOKE OK 29.2ms
(ServeReplica:my_application:main_agent pid=32678) INFO 2024-10-17 09:20:04,534 my_application_main_agen

(ServeReplica:my_application:runner pid=32679) yellow!
(ServeReplica:my_application:runner pid=32679) blue


(ServeReplica:my_application:tool1 pid=32675) INFO 2024-10-17 09:20:12,043 my_application_tool1 mp0j7vsa 3069347b-4e5a-429f-804c-6d12b3b9c059 / replica.py:373 - AINVOKE OK 1.7ms
(ServeReplica:my_application:main_agent pid=32677) INFO 2024-10-17 09:20:12,047 my_application_main_agent e9tf3qs6 3069347b-4e5a-429f-804c-6d12b3b9c059 / replica.py:373 - AINVOKE OK 1.7ms
(ServeReplica:my_application:runner pid=32679) INFO 2024-10-17 09:20:12,048 my_application_runner 04ga38ej 3069347b-4e5a-429f-804c-6d12b3b9c059 / replica.py:373 - __CALL__ OK 11.7ms


In [8]:
!serve status

proxies:
  621cd835144bc39d9aa0e8a551edb67cb5a896d1d10f011b7b6f7cf4: HEALTHY
applications:
  my_application:
    status: RUNNING
    message: ''
    last_deployed_time_s: 1729153189.953881
    deployments:
      tool1:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      tool2:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      main_agent:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 2
        message: ''
      runner:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
target_capacity: null


In [10]:
result = await app.ainvoke.remote("yellow!")
result

2024-10-17 09:20:04,490	INFO handle.py:126 -- Created DeploymentHandle '6vyqrcod' for Deployment(name='runner', app='my_application').


'Hello! I was provided with some tool outputs: Tool 2 has been triggered.'

In [11]:
# we can also interact with the application via REST

import requests

response = requests.post("http://localhost:8000/", data="blue")

print(response.text)

Hello! I was provided with some tool outputs: Tool 1 has been triggered.
